Источники:

[Дообучение модели ASR](https://huggingface.co/learn/audio-course/ru/chapter5/fine-tuning)
[Тонкая настройка Whisper для многоязычного ASR с помощью Hugging Face Transformers](https://habr.com/ru/articles/783128/)

За основу возьмём модель whisper-large-v3.

Предобученная модель содержит 1550 M параметров, для обучения требуется видеокарта ~10 GB (Required VRAM).

Коллаб с протестированным кодом
https://colab.research.google.com/drive/1sjIZLpMk0__bSgG8S3TFNpiWdMEd7RNj?usp=sharing

In [1]:
# ШАГ 1: Подготовка окружения

# Если работаем в Google Collab, требуется загрузить библиотеки с помощью менеджера пакетов pip.
# $ pip install --upgrade pip
# $ pip install --upgrade datasets transformers accelerate soundfile librosa evaluate jiwer tensorboard gradio

from huggingface_hub import notebook_login

notebook_login()

# Пишем наш токен с huggingface, его можно сгенерировать здесь: https://huggingface.co/settings/tokens

ModuleNotFoundError: No module named 'huggingface_hub'

In [ ]:
# ШАГ 2: Загрузка набора данных

# Дообучение можно провести на датасете mozilla-foundation/common_voice_17_0 (последняя версия).
# 

from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset(
    "mozilla-foundation/common_voice_17_0", "ru", split="train", trust_remote_code=True
)
common_voice["validation"] = load_dataset(
    "mozilla-foundation/common_voice_17_0", "ru", split="validation", trust_remote_code=True
)
common_voice["test"] = load_dataset(
    "mozilla-foundation/common_voice_17_0", "ru", split="test", trust_remote_code=True
)

print(common_voice)

In [ ]:
# Выбираем колонки с аудио и соответствующим текстом. В common_voice есть так же accent и locale — их игнорируем.

common_voice = common_voice.select_columns(["audio", "sentence"])


# Итоговый датасет для дообучения содержит следующие данные:
#DatasetDict({
#    train: Dataset({
#       features: ['audio', 'sentence'],
#        num_rows: 26377
#    })
#    validation: Dataset({
#        features: ['audio', 'sentence'],
#        num_rows: 10203
#    })
#    test: Dataset({
#        features: ['audio', 'sentence'],
#        num_rows: 10203
#    })
#})

In [ ]:
# ПАЙПЛАЙН
# Задача распознавания речи требует выполнения трёх этапов:
# 1) Извлечение признаков. На вход подаётся сырой аудиосигнал, из него выводятся логарифмические мел-спектрограммы
# 2) Модель sequence-to-sequence
# 3) Токенизатор. Выполняется постобработка предсказанных токенов в текст.

# ШАГ 1. ПОДГОТОВКА ДАННЫХ
# Для дообучения нужно взять предварительно обученную контрольную точку модели на русском языке.

from transformers import WhisperProcessor # импортируем класс WhisperProcessor, который содержит в себе извлекатель признаков WhisperFeatureExtractor и токенизатор WhisperTokenizer

processor = WhisperProcessor.from_pretrained(
    "openai/whisper-large-v3", language="russian", task="transcribe"
)

# Проверяем данные наших аудиоданных.

common_voice["train"].features

## Вывод:
#{'audio': Audio(sampling_rate=48000, mono=True, decode=True, id=None),
# 'sentence': Value(dtype='string', id=None)}

# Если частота дискретизации не равна 16кГц, следует это исправить:
#from datasets import Audio
#sampling_rate = processor.feature_extractor.sampling_rate
#common_voice = common_voice.cast_column("audio", Audio(sampling_rate=sampling_rate))

# ПОДГОТОВКА ДАННЫХ ДЛЯ МОДЕЛИ

def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=example["sentence"],
    )
    # Вычисляем длину аудиопримера в секундах, деля количество сэмплов (длину массива аудио) на частоту дискретизации.
    # Результат сохраняется в 'example' под ключом 'input_length'.
    example["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return example

# Удаляем столбцы с аудио и текстом их датасета, оставляем только те столбцы, что вернула функция prepare_dataset()
common_voice = common_voice.map(
    prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1
)

# Максимальная длина аудиофрагмента = 30 секунд.
max_input_length = 30.0

# Фильтруем аудио по длине (тем, что длиннее 30 сек, присваиваем значение False)
def is_audio_in_length_range(length):
    return length < max_input_length

# Применяем функции фильтрации по длине ко всем образцам обучающего набора данных
common_voice["train"] = common_voice["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
)

# Проверка, сколько обучающих данных удалили:
#common_voice["train"]

In [ ]:
# ШАГ 2: Обучение модели

# Определение сборщика данных

import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # Разделяем входные данные и метки, так как они имеют разные длины и требуют различных методов дополнения
        # Сначала обрабатываем аудиовходы, возвращая тензоры torch
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Получаем токенизированные последовательности меток
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # Дополняем метки до максимальной длины
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # Заменяем дополнение на -100, чтобы правильно игнорировать потерю
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # Если токен начала последовательности (bos) был добавлен на предыдущем этапе токенизации,
        # убираем токен bos здесь, так как он будет добавлен позже
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

# Инициализируем сборщик данных.
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

# Определяем метрику оценки.
# Используем Word Error Rate (WER).
# Загружаем метрику из библиотеки:

import evaluate
metric = evaluate.load("wer")

In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Заменяем -100 на идентификатор токена дополнения (pad_token_id)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # Мы не хотим группировать токены при вычислении метрик
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # Вычисляем орфографическую WER (Word Error Rate)
    wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

    # Вычисляем нормализованную WER
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]
    
    # Этап фильтрации, чтобы оценивать только те образцы, которые соответствуют ненулевым ссылкам:
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]

    wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm) # вычисление нормализованной оценки ошибки слов (Word Error Rate, WER) для предсказанных значений

    return {"wer_ortho": wer_ortho, "wer": wer}

In [ ]:
# Загружаем предобученную контрольную точку Whisper

from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3")


# Переопределим два аргумента для генерации, чтобы контролировать поведение модели во время вывода:
# мы принудительно зададим токены языка и задачи во время генерации, установив аргументы language и task,
# и также снова включим кэш для генерации, чтобы ускорить время вывода:
from functools import partial

# Отключаем кэширование во время обучения, так как это несовместимо с контрольной точкой градиента
model.config.use_cache = False

# Устанавливаем язык и задачу для генерации и повторно включаем кэш
model.generate = partial(
    model.generate, language="sinhalese", task="transcribe", use_cache=True
)

In [ ]:
# Определение конфигурации обучения

# Установим 500 шагов обучения. !Увеличим до 4000, есть собственный GPU или платный план Colab!

from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-large-v3-rus",  # name on the HF Hub
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # увеличьте в 2 раза при каждом уменьшении размера batch в 2 раза
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=500,  # увеличьте до 4000, если у вас есть собственный GPU или платный план Colab. Также рекомендуем изменить планировщик скорости обучения на линейный (установите lr_scheduler_type="linear"), так как это также принесет дополнительный прирост производительности при длительных обучающихся сессиях.
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [ ]:
# Передаем аргументы обучения в Trainer вместе с нашей моделью, набором данных, сборщиком данных и функцией compute_metrics:

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

In [ ]:
# ОБУЧЕНИЕ МОДЕЛИ

trainer.train()

In [ ]:
# Загружаем результаты обучения на Hugging Face Hub
trainer.push_to_hub(**kwargs)

# Дообученная контрольная точка готова.
# Любой пользователь может вызвать её:

#from transformers import pipeline

#pipe = pipeline("automatic-speech-recognition", model="USERNAME/whisper-large-v3-rus")